In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
THRESH_SPO2_LOW = 92
THRESH_SPO2_CRITICAL = 88
THRESH_SHOCK_INDEX_WARNING = 0.9
THRESH_SHOCK_INDEX_CRITICAL = 1.0
THRESH_MAP_LOW = 65  # Mean Arterial Pressure threshold for shock
THRESH_LACTATE_HIGH = 2.0  # mmol/L for tissue hypoperfusion
THRESH_LACTATE_CRITICAL = 4.0
THRESH_UO_LOW = 0.5  # mL/kg/hr (oliguria)

# Trend Analysis
TREND_WINDOW = 6  # Number of readings for short-term trend analysis


# AGE-SPECIFIC VITAL SIGN THRESHOLDS (Low, Normal, High)

AGE_THRESHOLDS = {
    'neonate': {
        'rr_low': 30, 'rr_normal': 40, 'rr_high': 60,
        'hr_low': 100, 'hr_normal': 140, 'hr_high': 160,
        'sbp_low': 60, 'sbp_normal': 70, 'sbp_high': 90,
        'temp_low': 36.0, 'temp_normal': 37.2, 'temp_high': 38.0
    },
    'infant': {
        'rr_low': 24, 'rr_normal': 30, 'rr_high': 40,
        'hr_low': 80, 'hr_normal': 120, 'hr_high': 140,
        'sbp_low': 70, 'sbp_normal': 85, 'sbp_high': 100,
        'temp_low': 36.0, 'temp_normal': 37.2, 'temp_high': 38.0
    },
    'child': {
        'rr_low': 16, 'rr_normal': 20, 'rr_high': 30,
        'hr_low': 70, 'hr_normal': 90, 'hr_high': 110,
        'sbp_low': 80, 'sbp_normal': 95, 'sbp_high': 110,
        'temp_low': 36.0, 'temp_normal': 37.0, 'temp_high': 38.0
    },
    'adolescent': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 20,
        'hr_low': 60, 'hr_normal': 75, 'hr_high': 100,
        'sbp_low': 90, 'sbp_normal': 105, 'sbp_high': 120,
        'temp_low': 35.8, 'temp_normal': 36.8, 'temp_high': 37.8
    },
    'adult': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 20,
        'hr_low': 60, 'hr_normal': 80, 'hr_high': 100,
        'sbp_low': 90, 'sbp_normal': 115, 'sbp_high': 130,
        'temp_low': 35.5, 'temp_normal': 36.8, 'temp_high': 38.0
    },
    'geriatric': {  # Added for elderly patients who may have different baselines
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 24, # Often higher RR baseline
        'hr_low': 55, 'hr_normal': 70, 'hr_high': 90, # Often lower HR
        'sbp_low': 90, 'sbp_normal': 125, 'sbp_high': 140, # Often higher SBP
        'temp_low': 35.5, 'temp_normal': 36.5, 'temp_high': 37.5 # Often lower temp
    }
}

In [3]:
def assign_age_category(df):
    """
    Assigns an age category based on the 'age' column in the DataFrame.
    Now includes more granular pediatric categories and a geriatric category.
    """
    df = df.copy()
    
    def _categorize(age):
        if age <= 0.083: return 'neonate'     # < 1 month
        elif age <= 1:   return 'infant'      # 1 month - 1 year
        elif age < 5:    return 'child'       # 1 - 5 years
        elif age < 13:   return 'adolescent'  # 5 - 12 years
        elif age < 65:   return 'adult'       # 13 - 64 years
        else:            return 'geriatric'   # 65+ years
    
    if 'age' in df.columns:
        df['age_category'] = df['age'].apply(_categorize)
    else:
        # Default to adult if age is not provided
        df['age_category'] = 'adult'
    
    return df

In [4]:
def apply_vital_range_flags(df, window_hours=12, sustain_threshold=0.5):
    """
    Apply standard vital range flags + thyroid storm concern.
    Works with your unified flagging system.

    Args:
        df: DataFrame with columns ['timestamp','temperature','heart_rate','sbp',...]
        window_hours: evaluation window (default 12h)
        sustain_threshold: fraction threshold (default 0.5 = 50%)

    Returns:
        DataFrame with new flag columns + thyroid storm status
    """
    df = df.copy().sort_values("timestamp")

    # --- Standard flags you already had ---
    df['flag_spo2_low'] = df['spo2'] < 92
    df['flag_temp_high'] = df['temperature'] >= 38.0
    df['flag_temp_low']  = df['temperature'] <= 36.0
    df['flag_rr_high']   = df['resp_rate'] >= 22
    df['flag_hr_high']   = df['heart_rate'] >= 100
    df['flag_sbp_low']   = df['sbp'] <= 100

    # --- Thyroid Storm Concern (sustained fever + tachycardia ± BP swings) ---
    # restrict to window
    if 'timestamp' in df.columns:
        cutoff = df['timestamp'].max() - timedelta(hours=window_hours)
        df_window = df[df['timestamp'] >= cutoff]
    else:
        df_window = df

    thyroid_storm_flag = False
    thyroid_storm_high_concern = False
    status = "No Concern"

    if not df_window.empty:
        # sustained fever
        fever_frac = (df_window['temperature'] >= 39.0).mean()
        fever_flag = fever_frac >= sustain_threshold

        # sustained tachycardia
        tachy_frac = (df_window['heart_rate'] >= 140).mean()
        tachy_flag = tachy_frac >= sustain_threshold

        # BP swings
        if 'sbp' in df_window.columns and df_window['sbp'].notna().any():
            sbp_range = df_window['sbp'].max() - df_window['sbp'].min()
            sbp_low_frac = (df_window['sbp'] < 90).mean()
            bp_swing_flag = (sbp_range > 30) or (sbp_low_frac >= 0.2)
        else:
            sbp_range, sbp_low_frac, bp_swing_flag = None, None, False

        # combine
        thyroid_storm_flag = fever_flag and tachy_flag
        thyroid_storm_high_concern = thyroid_storm_flag and bp_swing_flag

        if thyroid_storm_high_concern:
            status = "High Concern"
        elif thyroid_storm_flag:
            status = "Concern"

        # Save evidence in the df metadata
        df.attrs['thyroid_storm_evidence'] = {
            "fever_fraction": round(fever_frac, 2),
            "tachy_fraction": round(tachy_frac, 2),
            "sbp_range": round(sbp_range, 1) if sbp_range is not None else None,
            "sbp_low_fraction": round(sbp_low_frac, 2) if sbp_low_frac is not None else None,
        }

    # add columns
    df['flag_thyroid_storm'] = thyroid_storm_flag
    df['flag_thyroid_storm_high'] = thyroid_storm_high_concern
    df['thyroid_storm_status'] = status

    return df


In [5]:
def compute_recent_trends_delta(df):
    """
    Computes trends for each vital by differencing consecutive readings.
    Applies stricter interpretation using age-specific thresholds.
    """
    df = df.copy().sort_values("timestamp").reset_index(drop=True)

    if 'age_category' not in df.columns:
        df = assign_age_category(df)

    trends = {}
    recent = df.tail(TREND_WINDOW)
    age_group = recent['age_category'].iloc[-1]
    thresholds = AGE_THRESHOLDS[age_group]

    vital_map = {
        'resp_rate': ('rr_low', 'rr_normal', 'rr_high'),
        'heart_rate': ('hr_low', 'hr_normal', 'hr_high'),
        'sbp': ('sbp_low', 'sbp_normal', 'sbp_high'),
        'temperature': ('temp_low', 'temp_normal', 'temp_high'),
        'spo2': (None, None, None)  # handled separately
    }

    for vital in ['resp_rate', 'heart_rate', 'sbp', 'temperature', 'spo2']:
        if vital not in recent.columns or recent[vital].isnull().all():
            continue

        y = recent[vital].dropna().values
        if len(y) < 2:
            continue

        avg_delta = np.mean(np.diff(y))
        latest = y[-1]
        trends[f"{vital}_trend"] = round(avg_delta, 3)

        if vital == 'spo2':
            if latest < THRESH_SPO2_LOW:
                if avg_delta > 0:
                    flag = "Still abnormal — but improving"
                elif avg_delta < 0:
                    flag = "Abnormal and worsening"
                else:
                    flag = "Abnormal and flat"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        else:
            low_key, norm_key, high_key = vital_map[vital]
            low = thresholds[low_key]
            normal = thresholds[norm_key]
            high = thresholds[high_key]

            if latest < low or latest > high:
                if (latest > high and avg_delta < 0) or (latest < low and avg_delta > 0):
                    flag = "Still abnormal — but improving"
                else:
                    flag = "Abnormal and worsening"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        trends[f"{vital}_trend_flag"] = flag

    # Shock Index trend
    if all(col in recent.columns for col in ['heart_rate', 'sbp']):
        hr = recent['heart_rate'].values
        sbp = np.clip(recent['sbp'].values, a_min=1, a_max=None)
        si = hr / sbp

        if len(si) >= 2:
            avg_si_delta = np.mean(np.diff(si))
            trends['shock_index_trend'] = round(avg_si_delta, 3)

            latest_si = si[-1]
            if latest_si >= THRESH_SHOCK_INDEX_WARNING:
                flag = "Shock Index high — improving" if avg_si_delta < 0 else "Shock Index high — worsening"
            else:
                flag = "Normal but improving" if avg_si_delta < 0 else "Normal but rising"

            trends['shock_index_trend_flag'] = flag

    return trends

In [6]:
def detect_thyroid_storm_concern(df, window_hours=12, sustain_threshold=0.5):
    """
    Detects possible thyroid storm based on sustained fever + tachycardia (± BP swings).

    Criteria:
        • Temp ≥ 39 °C for ≥ 50% of last 12 h
        • HR ≥ 140 bpm for ≥ 50% of last 12 h
        • BP swings: SBP variability > 30 mmHg or recurrent hypotension
    
    Args:
        df: DataFrame with columns ['timestamp', 'temperature', 'heart_rate', 'sbp']
        window_hours: time window to evaluate (default 12h)
        sustain_threshold: fraction of time criteria must be met (default 0.5 = 50%)

    Returns:
        dict with flags, status, and evidence
    """
    df = df.copy().sort_values("timestamp")

    # restrict to window
    if 'timestamp' in df.columns:
        cutoff = df['timestamp'].max() - timedelta(hours=window_hours)
        df = df[df['timestamp'] >= cutoff]

    if df.empty:
        return {"thyroid_storm_flag": False, "status": "insufficient data"}

    n = len(df)

    # sustained fever
    fever_frac = (df['temperature'] >= 39.0).mean()
    fever_flag = fever_frac >= sustain_threshold

    # sustained tachycardia
    tachy_frac = (df['heart_rate'] >= 140).mean()
    tachy_flag = tachy_frac >= sustain_threshold

    # BP swings
    if 'sbp' in df.columns and df['sbp'].notna().any():
        sbp_range = df['sbp'].max() - df['sbp'].min()
        sbp_low_frac = (df['sbp'] < 90).mean()
        bp_swing_flag = (sbp_range > 30) or (sbp_low_frac >= 0.2)  # ≥20% hypotensive
    else:
        bp_swing_flag = False

    # combine
    concern_flag = fever_flag and tachy_flag
    high_concern_flag = concern_flag and bp_swing_flag

    # output
    return {
        "thyroid_storm_flag": concern_flag,
        "thyroid_storm_high_concern": high_concern_flag,
        "status": "High Concern" if high_concern_flag else (
            "Concern" if concern_flag else "No Concern"
        ),
        "evidence": {
            "fever_fraction": round(fever_frac, 2),
            "tachy_fraction": round(tachy_frac, 2),
            "sbp_range": round(sbp_range, 1) if 'sbp' in df.columns else None,
            "sbp_low_fraction": round(sbp_low_frac, 2) if 'sbp' in df.columns else None
        }
    }


In [7]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Create timestamps every hour for 12 hours
timestamps = [datetime.now() - timedelta(hours=i) for i in reversed(range(12))]

# Synthetic vitals
temperature = [38.5, 39.0, 39.2, 39.1, 39.5, 39.3, 39.0, 39.2, 39.4, 39.1, 38.9, 39.0]  # mostly ≥39
heart_rate = [135, 138, 142, 145, 148, 140, 143, 141, 146, 150, 139, 142]               # mostly ≥140
sbp = [120, 115, 118, 90, 95, 125, 110, 100, 105, 130, 115, 120]                        # some BP swings

# Optional columns for your pipeline
resp_rate = [18]*12
spo2 = [96]*12
age = [35]*12  # adult

# Construct DataFrame
df_test = pd.DataFrame({
    'timestamp': timestamps,
    'temperature': temperature,
    'heart_rate': heart_rate,
    'sbp': sbp,
    'resp_rate': resp_rate,
    'spo2': spo2,
    'age': age
})

result = detect_thyroid_storm_concern(df_test)
print(result)


{'thyroid_storm_flag': np.True_, 'thyroid_storm_high_concern': np.True_, 'status': 'High Concern', 'evidence': {'fever_fraction': np.float64(0.83), 'tachy_fraction': np.float64(0.75), 'sbp_range': np.int64(40), 'sbp_low_fraction': np.float64(0.0)}}


In [8]:
df_flagged = apply_vital_range_flags(df_test)
print(df_flagged[['temperature','heart_rate','sbp','flag_thyroid_storm','flag_thyroid_storm_high','thyroid_storm_status']])


    temperature  heart_rate  sbp  flag_thyroid_storm  flag_thyroid_storm_high  \
0          38.5         135  120                True                     True   
1          39.0         138  115                True                     True   
2          39.2         142  118                True                     True   
3          39.1         145   90                True                     True   
4          39.5         148   95                True                     True   
5          39.3         140  125                True                     True   
6          39.0         143  110                True                     True   
7          39.2         141  100                True                     True   
8          39.4         146  105                True                     True   
9          39.1         150  130                True                     True   
10         38.9         139  115                True                     True   
11         39.0         142 